In [1]:
import json

In [2]:
with open('output/levenshtein_dict.json','r') as out:
    for line in out:
        data_1 = json.loads(line)

In [3]:
with open('output/levenshtein_dict.json','r') as out:
    for line in out:
        data_2 = json.loads(line)

In [4]:
print(len(data_1),len(data_2))

(401, 401)


In [5]:
filterTrue1 = [d for d in data_1 if d['is_correct'] == True]
filterFalse1 = [d for d in data_1 if d['is_correct'] == False]
print(len(filterTrue1), len(filterFalse1))

(253, 148)


In [6]:
filterTrue2 = [d for d in data_2 if d['is_correct'] == True]
filterFalse2 = [d for d in data_2 if d['is_correct'] == False]
print(len(filterTrue2), len(filterFalse2))

(255, 146)


In [15]:
with open('output/jaro_winkler_words.json','r') as out:
    for line in out:
        data_3 = json.loads(line)

In [18]:
with open('output/lavenshtein_words.json','r') as out:
    for line in out:
        data_4 = json.loads(line)

In [19]:
print(len(data_3), len(data_4))

(101, 101)


In [20]:
filterTrue3 = [d for d in data_3 if d['is_correct'] == True]
filterFalse3 = [d for d in data_3 if d['is_correct'] == False]
print(len(filterTrue3), len(filterFalse3))

(49, 52)


In [21]:
filterTrue4 = [d for d in data_4 if d['is_correct'] == True]
filterFalse4 = [d for d in data_4 if d['is_correct'] == False]
print(len(filterTrue4), len(filterFalse4))

(71, 30)
